# 生成所需数据

In [1]:
import os
import pandas as pd
import numpy as np
import shutil
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *

In [14]:
def add_and_calc_field(in_feature, source_field, target_field, field_type='TEXT'):
    """
    添加新字段并根据源字段计算值
    
    Parameters:
    -----------
    in_feature : str
        输入要素路径
    source_field : str
        源字段名称
    target_field : str
        目标字段名称（新增字段）
    field_type : str, optional
        新增字段类型，默认为'TEXT'
    """
    # 添加新字段
    arcpy.AddField_management(in_table=in_feature, 
                            field_name=target_field,
                            field_type=field_type)
    
    # 定义计算表达式（单行形式）
    code_block = """
def get_value(val):
    if val is None:
        return 1
    try:
        map_dict = {1:1, 2:1, 3:2, 4:2, 5:3, 6:3}
        return map_dict.get(int(val), 1)
    except:
        return 1
"""
    expression = f"get_value(!{source_field}!)"
    
    # 使用字段计算工具更新字段值
    arcpy.CalculateField_management(in_table=in_feature,
                                  field=target_field,
                                  expression=expression,
                                  expression_type="PYTHON3",
                                  code_block=code_block)

In [8]:
# 指定工作空间
env.workspace = r'F:\cache_data\shp_file\ky\ele_ky'
if not os.path.exists(env.workspace):
    os.makedirs(env.workspace)
env.overwriteOutput = True

In [9]:
# 三调地块路径
sd_data = r'F:\cache_data\shp_file\ky\ky_sd_polygon.shp'
# 母质路径
mz_data = r'F:\cache_data\shp_file\ky\dz\dz_dissove_20250228.shp'
# 坡位矢量路径
slope_data = r'F:\cache_data\shp_file\ky\ky_slopepostion_smooth.shp'

In [ ]:
# 更新坡位
add_and_calc_field(slope_data,'gridcode','slope')

In [17]:
# 融合坡位
dissolve_name = 'ky_slopepostion_dissove.shp'
arcpy.analysis.PairwiseDissolve(
    in_features=slope_data,
    out_feature_class=dissolve_name,
    dissolve_field=['slope'],
    statistics_fields=None,
    multi_part="SINGLE_PART",
    concatenation_separator=""
)

<Result 'F:\\cache_data\\shp_file\\ky\\ele_ky\\ky_slopepostion_dissove.shp'>

In [18]:
# 提取耕林园草
select_name = 'ky_sd_select.shp'
arcpy.Select_analysis(sd_data,select_name,f"{'DLLB'} IN ('自然土','水稻土', '非自然土')")

<Result 'F:\\cache_data\\shp_file\\ky\\ele_ky\\ky_sd_select.shp'>

In [20]:
# 标识
intersect_name = 'ky_merge_data.shp'
# 使用标识工具而非相交，保持第一个数据集的完整性
# 第一步：先对第一个和第二个数据集执行标识
temp_identity_name = 'ky_identity_temp.shp'
arcpy.Identity_analysis(in_features=select_name, 
                      identity_features=mz_data, 
                      out_feature_class=temp_identity_name,
                      join_attributes='ALL')

# 第二步：对中间结果和第三个数据集执行标识
arcpy.Identity_analysis(in_features=temp_identity_name, 
                      identity_features=dissolve_name, 
                      out_feature_class=intersect_name,
                      join_attributes='ALL')

<Result 'F:\\cache_data\\shp_file\\ky\\ele_ky\\ky_merge_data.shp'>

In [21]:
# 转单部件
single_name = 'ky_merge_data_single.shp'
arcpy.MultipartToSinglepart_management(intersect_name,single_name)

<Result 'F:\\cache_data\\shp_file\\ky\\ele_ky\\ky_merge_data_single.shp'>